# Stream Parquet Files to Delta Table
This notebook reads Parquet files as a stream and writes them to a Delta table with schema handling.

In [ ]:
# Configure spark to handle schema merging
spark.conf.set("spark.sql.streaming.schemaInference", True)
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", True)

In [ ]:
# Define the input and output paths
input_path = "abfss://external-location@databricksdevstgacc.dfs.core.windows.net/files2/"
output_table = "dev_catalog.default.stream_test"

In [ ]:
# Read the streaming data
stream_df = spark.readStream \
    .format("parquet") \
    .option("mergeSchema", "true") \
    .load(input_path)

In [ ]:
# Write the streaming data to Delta table
stream_query = stream_df.writeStream \
    .format("delta") \
    .option("checkpointLocation", "_checkpoint/stream_test") \
    .outputMode("append") \
    .trigger(processingTime='1 minute') \
    .table(output_table)

In [ ]:
# Wait for the streaming query to finish
stream_query.awaitTermination()